In [ ]:
import pycuda.autoinit
import numpy as np
from pycuda import gpuarray
import pycuda.driver as drv
import pandas as pd
from pycuda.compiler import SourceModule
import cv2
import itertools

In [ ]:
# Compile cuda code
module = SourceModule(open("kernels_raytracer.cu", "r").read())
kernel_draw_scene = module.get_function("draw_scene")

In [ ]:
kernel_hello = module.get_function("hello_world")
grid_size = 1
block = (1, 1, 1)
grid = (grid_size, grid_size)
kernel_hello(block=block, grid=grid)

In [ ]:
# !!!! This is stolen! Consider rewritting. !!!!
def stack_padding(it):

    def resize(row, size):
        new = np.array(row)
        new.resize((len(row), size))
        return new

    row_length = max([x.shape[1] for x in it])
    return np.concatenate( [resize(row, row_length) for row in it] )

In [ ]:
# Define spheres
# Format is x,y,z,radius,r,g,b
spheres_params = np.array([[1, 0, 0, 100, 300, 10, 50],
                           [0, 0, 1, 200, 30, 30, 50]], dtype=np.float32)
n_spheres = len(spheres_params)
param_size_sphere = spheres_params[0].nbytes
#spheres_params_gpu = gpuarray.to_gpu(spheres_params)

In [ ]:
# Combine object types
n_types = 1

object_params = stack_padding([spheres_params])
#object_params = np.array([np.uintp(spheres_params_gpu.ptr)], dtype=np.uintp)
object_params_gpu = gpuarray.to_gpu(object_params)

num_objs = np.array([n_spheres], dtype=np.int32)
num_objs_gpu = gpuarray.to_gpu(num_objs)

n_params = object_params.shape[1]
#param_size = np.array([param_size_sphere], dtype=np.int32)
#param_size_gpu = gpuarray.to_gpu(param_size)

In [ ]:
# Define image
dim = 512
#dim = 4
bitmap = np.zeros((dim, dim, 3), dtype=np.int32)
bitmap_gpu = gpuarray.to_gpu(bitmap)

In [ ]:
# Calculate the bitmap
grid_size = 16
#grid_size = 4
block = (dim//grid_size, dim//grid_size, 1)
grid = (grid_size, grid_size)

kernel_draw_scene(bitmap_gpu, 
                  np.int32(dim), 
                  object_params_gpu,
                  num_objs_gpu,
                  np.int32(n_params),
                  np.int32(n_types),
                  block=block,
                  grid=grid)

In [ ]:
image = cv2.cvtColor(bitmap_gpu.get().astype(np.uint8), cv2.COLOR_RGB2BGR)
cv2.imwrite('color_img.jpg', image)